In [353]:
import pandas as pd

corpus = pd.read_csv("./corpus_english.csv", encoding='utf-8')
corpus.head()

,sentences
0,Cannot save files offline because Google drive...
1,This is the worst app I have seen before when ...
2,I cant open this app i dont know why
3,Cannot install the app. It started as not bein...
4,Doesnt save the file I edit and make me waste ...


In [354]:
corpus['sentences'][50]

'Super lag I cant even sign in 😡it also crash every time I open it fix it🤬'

In [355]:
import re

def delete_emojis(texto):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # Emoticonos
                               u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
                               u"\U0001F680-\U0001F6FF"  # Transporte y mapa de caracteres
                               u"\U0001F700-\U0001F77F"  # Símbolos de alquimia
                               u"\U0001F780-\U0001F7FF"  # Símbolos de alquimia suplementarios
                               u"\U0001F800-\U0001F8FF"  # Símbolos suplementarios para la escritura
                               u"\U0001F900-\U0001F9FF"  # Emoticonos y objetos
                               u"\U0001FA00-\U0001FA6F"  # Símbolos de ajedrez
                               u"\U0001FA70-\U0001FAFF"  # Símbolos de ajedrez suplementarios
                               u"\U00002702-\U000027B0"  # Otros emoticonos
                               u"\U000024C2-\U0001F251" 
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', texto)

In [356]:
corpus['new_sentences'] = corpus['sentences'].apply(delete_emojis)

In [357]:
corpus['new_sentences'][50]

'Super lag I cant even sign in it also crash every time I open it fix it'

In [358]:
corpus = corpus.drop('sentences', axis=1)
corpus = corpus.rename(columns={'new_sentences': 'review'})
corpus.head()

,review
0,Cannot save files offline because Google drive...
1,This is the worst app I have seen before when ...
2,I cant open this app i dont know why
3,Cannot install the app. It started as not bein...
4,Doesnt save the file I edit and make me waste ...


In [359]:
import re

url_pattern = re.compile(r'https?://\S+')
def remove_urls(text):
    return url_pattern.sub('', text)

corpus['review'] = corpus['review'].apply(remove_urls)

In [360]:
corpus['review'] = corpus['review'].str.lower()
corpus.head()

,review
0,cannot save files offline because google drive...
1,this is the worst app i have seen before when ...
2,i cant open this app i dont know why
3,cannot install the app. it started as not bein...
4,doesnt save the file i edit and make me waste ...


In [361]:
corpus = corpus.replace(to_replace=r'[^\w\s]', value='', regex=True)

In [362]:
corpus = corpus.replace(to_replace=r'\d', value='', regex=True)

In [363]:
import nltk
from nltk.tokenize import word_tokenize

corpus['review'] = corpus['review'].apply(word_tokenize)

In [364]:
corpus.head()

,review
0,"[can, not, save, files, offline, because, goog..."
1,"[this, is, the, worst, app, i, have, seen, bef..."
2,"[i, cant, open, this, app, i, dont, know, why]"
3,"[can, not, install, the, app, it, started, as,..."
4,"[doesnt, save, the, file, i, edit, and, make, ..."


In [365]:
def convert_contractions(tokens):
    contraction_dict = {
        "cant": "can",
        "doesnt": "does",
        "dont": "do",
        "couldnt": "could",
        "isnt": "is",
        "wasnt": "was",
        "werent": "were",
        "hasnt": "has",
        "havent": "have",
        "hadnt": "had",
        "wont": "will",
        "wouldnt": "would",
        "arent": "are",
        "didnt": 'did not',
        "idk": 'i do know',
        "couldntve": "could not have",
        "im": " ",
        "hw": " ",
        "na": " ",
        "please": " ",
        "pls": " ",
        "plss": " ",
        "ive": " ",
        "tssssss": " ",

    }

    converted_tokens = [contraction_dict[token] if token in contraction_dict else token for token in tokens]
    return converted_tokens
    
corpus['review'] = corpus['review'].apply(convert_contractions)
corpus.head()

,review
0,"[can, not, save, files, offline, because, goog..."
1,"[this, is, the, worst, app, i, have, seen, bef..."
2,"[i, can, open, this, app, i, do, know, why]"
3,"[can, not, install, the, app, it, started, as,..."
4,"[does, save, the, file, i, edit, and, make, me..."


In [366]:
corpus = corpus.replace(to_replace=r'[^\w\s]', value='', regex=True)

In [367]:
import nltk
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
corpus['review'] = corpus['review'].apply(lambda x: [word for word in x if word not in stop_words])

In [368]:
corpus.head()

,review
0,"[save, files, offline, google, drive, date, la..."
1,"[worst, app, seen, logging, see, sorry, give, ..."
2,"[open, app, know]"
3,"[install, app, started, able, update, uninstal..."
4,"[save, file, edit, make, waste, time, editing]"


In [369]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    def get_wordnet_pos(word):
        tag = nltk.pos_tag([word])[0][1][0].upper()
        tag_dict = {"J": wordnet.ADJ,
                    "N": wordnet.NOUN,
                    "V": wordnet.VERB,
                    "R": wordnet.ADV}
        return tag_dict.get(tag, wordnet.NOUN)

    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(token)) for token in tokens]
    return lemmas

corpus['review'] = corpus['review'].apply(lemmatize_tokens)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [370]:
from nltk.stem import PorterStemmer

nltk.download('punkt')

stemmer = PorterStemmer()   

# def stem_words(words):
#     return [stemmer.stem(word) for word in words]

def stem_words2(words):
    return [stemmer.stem(word) if word.endswith('ing') else word for word in words]

# corpus['review'] = corpus['review'].apply(stem_words)
corpus['review'] = corpus['review'].apply(stem_words2)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [371]:
corpus.head()

,review
0,"[save, file, offline, google, drive, date, lat..."
1,"[bad, app, see, log, see, sorry, give, access,..."
2,"[open, app, know]"
3,"[install, app, start, able, update, uninstal, ..."
4,"[save, file, edit, make, waste, time, edit]"


In [372]:
corpus['review_clean'] = corpus['review'].apply(lambda x: ' '.join(x))

In [373]:
corpus.head()

,review,review_clean
0,"[save, file, offline, google, drive, date, lat...",save file offline google drive date late version
1,"[bad, app, see, log, see, sorry, give, access,...",bad app see log see sorry give access help
2,"[open, app, know]",open app know
3,"[install, app, start, able, update, uninstal, ...",install app start able update uninstal try rei...
4,"[save, file, edit, make, waste, time, edit]",save file edit make waste time edit


In [374]:
import nltk
from nltk.tokenize import word_tokenize

corpus['review_clean'] = corpus['review_clean'].apply(word_tokenize)

In [375]:
corpus['review_clean'] = corpus['review_clean'].apply(lambda x: [word for word in x if word not in stop_words])

In [376]:
corpus['review_ready'] = corpus['review_clean'].apply(lambda x: ' '.join(x))

In [379]:
corpus

,review,review_ready
0,"[save, file, offline, google, drive, date, lat...",save file offline google drive date late version
1,"[bad, app, see, log, see, sorry, give, access,...",bad app see log see sorry give access help
2,"[open, app, know]",open app know
3,"[install, app, start, able, update, uninstal, ...",install app start able update uninstal try rei...
4,"[save, file, edit, make, waste, time, edit]",save file edit make waste time edit
...,...,...
12498,"[hate, app, data, package, fully, galli, consu...",hate app data package fully galli consume data...
12501,"[app, work, properly, show, access, classroom]",app work properly show access classroom
12502,"[open, loptop]",open loptop
12503,"[google, classroom, , fix, app, many, notific...",google classroom fix app many notification wor...


In [380]:
corpus['review_ready'].to_csv('./corpus_for_clustering.csv', index=False, header=['review'])